In [1]:
!pip install langchain langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.65
    Uninstalling langchain-core-0.3.65:
      Successfully uninstalled langchain-core-0.3.65
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.25
    Uninstalling langchain-0.3.25:
      Successfully uninstalled langchain-0.3.25


In [2]:
import os

In [3]:
os.environ['OPENAI_API_KEY'] = ""
os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""

In [4]:
!pip install openai

In [5]:
from langchain.llms import OpenAI
llm = OpenAI(temperature=0.9)

/tmp/ipython-input-5-166103413.py:2: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0.9)


In [6]:
text="What would be a good company name for a company that makes colorful socks?"

In [ ]:
print(llm.predict(text))

In [ ]:
print(llm(text))

In [ ]:
print(llm.invoke(text))

example2

In [9]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9)
name = llm.predict("I want to open a restaurant for Chinese food. Suggest a fency name for this.")
print(name)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
response=llm("I want to open a restaurant for Chinese food. Suggest a fency name for this.")
print(response)

HUgging face

In [13]:
pip install -U langchain-huggingface


In [10]:
!pip install huggingface_hub

In [14]:
from langchain_huggingface import HuggingFaceEndpoint


In [11]:
from langchain import HuggingFaceHub

In [ ]:
llm = HuggingFaceEndpoint(
    repo_id="google/flan-t5-large",
    temperature=0,
    max_length=64
)

response = llm.invoke("translate English to German: How old are you?")
print(response)

In [ ]:
from langchain import HuggingFaceHub

llm = HuggingFaceHub(repo_id="google/flan-t5-large", model_kwargs={"temperature":0, "max_length":64})
# name = llm.predict("I want to open a restaurant for Chinese food. Suggest a fency name for this.")
name = llm.predict("I want to open a restaurant for Indian food. Suggest a fency name for this.")
print(name)

prompt template

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)
p = prompt_template_name.format(cuisine="indian")
print(p)

chains

In [ ]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9)

In [ ]:
from langchain.prompts import PromptTemplate
prompt = PromptTemplate.from_template("What is a good name for a company that makes {product}")
prompt.format(product="colorful socks")

In [ ]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt)
response= chain.run("colorful socks")
print(response)

sequential chain

In [ ]:
llm = OpenAI(temperature=0.6)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="""Suggest some menu items for {restaurant_name}"""
)

food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items)

In [ ]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains = [name_chain, food_items_chain])

content = chain.run("indian")
print(content)

Sequential chain

In [ ]:
llm = OpenAI(temperature=0.7)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaurant_name")

In [ ]:
llm = OpenAI(temperature=0.7)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="Suggest some menu items for {restaurant_name}."
)

food_items_chain =LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")

In [ ]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', "menu_items"]
)

In [ ]:
print(chain({"cuisine": "indian"}))

Agents

In [ ]:
!pip install wikipedia

In [ ]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI

In [ ]:
llm = OpenAI(temperature=0)

In [ ]:
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)




agent.run("What was the GDP of US in 2024?")

Memory

In [ ]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9)

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

In [ ]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm,prompt=prompt_template_name)
name = chain.run("Mexican")
print(name)

In [ ]:
name = chain.run("Indian")
print(name)

In [ ]:
chain.memory

In [ ]:
type(chain.memory)

Converstaion buffer memory

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

chain = LLMChain(llm=llm, prompt=prompt_template_name, memory=memory)
name = chain.run("Mexican")
print(name)

In [ ]:
name = chain.run("Arabic")
print(name)

In [ ]:
print(chain.memory.buffer)

Conversation chain

just remember last 5 conversation chains

In [ ]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm=OpenAI(temperature=0.7))
print(convo.prompt.template)

In [ ]:
convo.run("Who won the first cricket world cup?")

In [ ]:
convo.run("How much is 5+5?")

In [ ]:
convo.run("Who was the captain of the winning team?")

In [ ]:
print(convo.memory.buffer)

Conversation buffer with window

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=3)

convo = ConversationChain(
    llm=OpenAI(temperature=0.7),
    memory=memory
)
convo.run("Who won the first cricket world cup?")

In [ ]:
convo.run("How much is 5+5?")

In [ ]:
convo.run("Who was the captain of the winning team?")

In [ ]:
print(convo.memory.buffer)